In [134]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard

np.set_printoptions(precision=6, suppress=True)

In [22]:


# Wprowadzenie ścieżek do danych
base_dir = '/content/drive/MyDrive/Studia/Semestr 6/Elementy Machine Learning/data/planes'
data_dir = '/content/drive/MyDrive/Studia/Semestr 6/Elementy Machine Learning/Dane_Kamil'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_rocket_dir = os.path.join(train_dir, 'rocket')
train_plane_dir = os.path.join(train_dir, 'plane')

valid_rocket_dir = os.path.join(valid_dir, 'rocket')
valid_plane_dir = os.path.join(valid_dir, 'plane')

test_rocket_dir = os.path.join(test_dir, 'rocket')
test_plane_dir = os.path.join(test_dir, 'plane')

# Tworzenie katalogów
for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_plane_dir, train_rocket_dir, valid_plane_dir, valid_rocket_dir, test_plane_dir, test_rocket_dir]

for directory in dirs:
    if not os.path.exists(directory):
        os.mkdir(directory)

# Filtrowanie i kopiowanie plików
rocket_fnames = [fname for fname in os.listdir(os.path.join(base_dir, 'rocket')) if fname.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']]
plane_fnames = [fname for fname in os.listdir(os.path.join(base_dir, 'passenger-plane')) if fname.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']]

size = min(len(rocket_fnames), len(plane_fnames))
train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

for i, fname in enumerate(rocket_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(train_rocket_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(valid_rocket_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(test_rocket_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(plane_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'passenger-plane', fname)
        dst = os.path.join(train_plane_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'passenger-plane', fname)
        dst = os.path.join(valid_plane_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'passenger-plane', fname)
        dst = os.path.join(test_plane_dir, fname)
        shutil.copyfile(src, dst)

print('rocket - zbiór treningowy', len(os.listdir(train_rocket_dir)))
print('rocket - zbiór walidacyjny', len(os.listdir(valid_rocket_dir)))
print('rocket - zbiór testowy', len(os.listdir(test_rocket_dir)))

print('plane - zbiór treningowy', len(os.listdir(train_plane_dir)))
print('plane - zbiór walidacyjny', len(os.listdir(valid_plane_dir)))
print('plane - zbiór testowy', len(os.listdir(test_plane_dir)))

# Funkcja do bezpiecznego ładowania obrazów
def safe_load_img(path, target_size):
    try:
        img = tf.keras.preprocessing.image.load_img(path, target_size=target_size)
        return img
    except Exception as e:
        print(f'Błąd ładowania obrazu: {path}. Błąd: {e}')
        return None

# Ustawienia generatorów danych
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1.0/255.0
)

valid_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
# Budowa modelu
model = Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))
model.summary()

model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Trenowanie modelu
log_dir = 'logs'
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)
os.mkdir(log_dir)

tensorboard = TensorBoard(log_dir=log_dir)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    callbacks=[tensorboard]
)

# Funkcja do rysowania wykresów historii treningu
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss')
    fig.show()

plot_hist(history)

rocket - zbiór treningowy 822
rocket - zbiór walidacyjny 234
rocket - zbiór testowy 117
plane - zbiór treningowy 822
plane - zbiór walidacyjny 234
plane - zbiór testowy 117
Found 1644 images belonging to 2 classes.
Found 468 images belonging to 2 classes.
Found 234 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                

In [23]:
# Zapisywanie modelu
model.save('rocket_plane_classifier_model.keras')

In [146]:
loaded_model = load_model('rocket_plane_classifier_model.keras')
data_dir = '/content/drive/MyDrive/Studia/Semestr 6/Elementy Machine Learning/Dane_Kamil/test'
planes_file_dir = os.listdir(os.path.join(data_dir, 'plane'))
print(planes_file_dir)
print(type(planes_file_dir))
rockets_file_dir = os.listdir(os.path.join(data_dir, 'rocket'))

planes_file_names = [fname for fname in planes_file_dir if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
rockets_file_names = [fname for fname in rockets_file_dir if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]

print(planes_file_names)
print(len(rockets_file_names))

def load_image(img_path, show=False):
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]
    return img_tensor

AttributeError: 'Sequential' object has no attribute 'classes_'

In [131]:
loaded_model.predict_function

In [142]:
def test_model(test_data_dir):
  # {'plane': 0, 'rocket': 1}
    count_0 = 0
    count_1 = 0
    all = 0
    print(test_data_dir)
    print(type(test_data_dir))
    for i, file_name in enumerate(test_data_dir):
          # print(file_name)
          # print(type(file_name))
          # print(planes_file_dir)
          # print(type(planes_file_dir))
        img_path = os.path.join('/content/drive/MyDrive/Studia/Semestr 6/Elementy Machine Learning/Dane_Kamil/test/plane', file_name)
        # print(img_path)
        img_tensor = load_image(img_path)
        pred = model.predict(img_tensor)
        pred_class = pred.argmax(-1)
        # print(pred_class)
        # print(img_path)
        # print(type(pred_class[0]))

        if (pred_class[0] == 0):
            count_0 += 1
        if (pred_class[0] == 1):
            count_1 += 1
        all += 1
    return (count_0, count_1, all)

In [157]:
def test_rocket_model(test_data_dir):
  # {'plane': 0, 'rocket': 1}
    count_0 = 0
    count_1 = 0
    all = 0
    # print(test_data_dir)
    # print(type(test_data_dir))
    for i, file_name in enumerate(test_data_dir):
        print(file_name)
          # print(type(file_name))
          # print(planes_file_dir)
          # print(type(planes_file_dir))
        img_path = os.path.join('/content/drive/MyDrive/Studia/Semestr 6/Elementy Machine Learning/Dane_Kamil/test/rocket', file_name)
        # print(img_path)
        img_tensor = load_image(img_path)
        pred = model.predict(img_tensor)
        pred_class = pred.argmax(-1)
        # print(pred_class)
        # print(img_path)
        # print(type(pred_class[0]))

        if (pred_class[0] == 0):
            count_0 += 1
        if (pred_class[0] == 1):
            count_1 += 1
        all += 1
    return (count_0, count_1, all)

In [158]:
#result_plane = test_model(planes_file_names)
predictions_rocket = test_rocket_model(rockets_file_names)

#print(predictions_plane)
print(predictions_rocket)
#print(result_plane)

00000062.jpg
1/1 [==============================] - 0s 25ms/step
00000077.jpg
1/1 [==============================] - 0s 16ms/step
00000074.jpg
1/1 [==============================] - 0s 16ms/step
00000067.jpg
1/1 [==============================] - 0s 16ms/step
00000075.jpg
1/1 [==============================] - 0s 18ms/step
00000083.jpg
1/1 [==============================] - 0s 16ms/step
00000086.jpg
1/1 [==============================] - 0s 17ms/step
00000071.jpg
1/1 [==============================] - 0s 16ms/step
00000064.jpeg
1/1 [==============================] - 0s 18ms/step
00000089.jpg
1/1 [==============================] - 0s 17ms/step
00000081.jpg
1/1 [==============================] - 0s 16ms/step
00000073.jpg
1/1 [==============================] - 0s 16ms/step
00000087.jpeg
1/1 [==============================] - 0s 17ms/step
00000065.jpg
1/1 [==============================] - 0s 27ms/step
00000079.jpg
1/1 [==============================] - 0s 17ms/step
00000060.jpg
1/1 [=====

In [147]:
print(train_generator.class_indices)

{'plane': 0, 'rocket': 1}
